Persona hub generates sample personas using documents from RedPajamav2 dataset: They prompt GPT4o with a document from the dataset and prompt it thus:

*Who is likely to read/like/dislike the following text? Be very detailed in your description of this persona*

They then use those personas to generate 'diverse' instructions. **But are the synthesized instructions actually that diverse?** Analyzing that in this notebook

In [31]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import ipdb
import re
from datasets import load_dataset
from diversity import compression_ratio, homogenization_score, ngram_diversity_score, extract_patterns, get_pos, pos_patterns, token_patterns
import json

import matplotlib.pyplot as plt
# import mplcursors
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=2)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)
np.set_printoptions(precision=3)

RandomState(MT19937) at 0x109CB7140

In [2]:
df = pd.read_json("hf://datasets/proj-persona/PersonaHub/knowledge.jsonl", lines=True)

In [3]:
df.shape

(10000, 3)

In [11]:
df['title'] = df['synthesized text'].apply(lambda x: x.split('\n\n')[0][7:])
df.head()

input persona  \
0                                                                                                                                                                           A high school teacher of social studies preparing lessons on the development of political parties in the United States.   
1                                                                                                                                                                                                              A geologist studying sunken volcanoes and the geological history of Pacific islands.   
2                                                                         A manuscript studies scholar or paleographer with expertise in early medieval European texts and handwriting. This expert would be intrigued by the ninth-century manuscript that contains copies of Theuthild's letters.   
3  I'm a legal scholar specializing in attorney conduct, courtroom behavior, and the First Amendment implications of disciplinary actions against lawyers. I'm interested in analyzing the Snyder case and its impact on attorneys' freedom of speech in their communications with court personnel.   
4                                                                                                                                                             A linguist specialized in the Khoekhoe language, also known as naq, with interest in endangered languages and language documentation.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

# How diverse are the titles?

In [18]:
titles = df['title'].values.tolist()
cr = compression_ratio(titles, 'gzip')
# hs = homogenization_score(titles, 'rougel')
# hs = homogenization_score(titles, 'bertscore') 
nds = ngram_diversity_score(titles, 4)

print(cr, nds)

3.238 2.204


Let's compare it to something standard as data - CNN/DailySum maybe?

The essays from Visakh/He He's paper showing that GPT-2 text is homogenous might have been a good start, but its too narrow and not big enough.

In [20]:
events = []
with open('../data/essays_solo.jsonl', 'r') as f:
    for event in f:
        events.append(json.loads(event))

In [23]:
essay_titles = [x['title'] for x in events]
cr = compression_ratio(essay_titles, 'gzip')
# hs = homogenization_score(titles, 'rougel')
# hs = homogenization_score(titles, 'bertscore') 
nds = ngram_diversity_score(essay_titles, 4)

print(cr, nds)

11.49 0.437


# How diverse are the generated knowledge base articles?

In [25]:
def clean_up(text):

    # remove title first
    text = " ".join(text.split('\n\n')[1:])

    return text

df['content'] = df['synthesized text'].apply(lambda x: clean_up(x))

In [28]:
contents = df['content'].values.tolist()
cr = compression_ratio(contents, 'gzip')
# hs = homogenization_score(data_example, 'rougel')
# hs = homogenization_score(data_example, 'bertscore') 
nds = ngram_diversity_score(contents, 4)

print(cr, nds)

3.06 1.72


In [30]:
essay_bodies = [x['essay'] for x in events]
cr = compression_ratio(essay_bodies, 'gzip')
# hs = homogenization_score(data_example, 'rougel')
# hs = homogenization_score(data_example, 'bertscore') 
nds = ngram_diversity_score(essay_bodies, 4)

print(cr, nds)

2.901 2.738


Even compared to the human written essays, the contents are more easily compresssed and have a higher n-gram diversity score. 

**What about POS CR?**

In [32]:
synt_patterns = token_patterns(contents, 5)

In [34]:
synt_patterns

[("In this article, I'll share", 3221),
 ("I've had the privilege of", 2548),
 ("this article, I'll share my", 2483),
 ("this article, I'll delve into", 1510),
 ("In this article, I'll delve", 1499),
 ('In this article, I will', 1485),
 ("article, I'll delve into the", 1404),
 ("article, I'll share my insights", 1254),
 ('I hope this article has', 1234),
 ('a deeper understanding of the', 1209)]

In [35]:
# get the POS patterns 
joined_pos, tuples = get_pos(contents)
ngrams_pos = token_patterns(joined_pos, 5, 10)

In [38]:
ngrams_pos

[('. LS LS NFP NFP', 38990),
 ('NN . LS LS NFP', 18479),
 ('LS LS NFP NFP NNP', 15855),
 ('NNS . LS LS NFP', 15820),
 ('NN , NN , CC', 15106),
 ('NNS , NNS , CC', 13133),
 ('NN , PRP MD VB', 12115),
 ('DT JJ NN IN DT', 12035),
 ('LS LS NFP NFP NN', 11149),
 ('IN DT NN , PRP', 10743)]

In [39]:
cr = compression_ratio(joined_pos, 'gzip')
# hs = homogenization_score(data_example, 'rougel')
# hs = homogenization_score(data_example, 'bertscore') 
nds = ngram_diversity_score(joined_pos, 4)

print(cr, nds)

6.089 0.02


In [41]:
# Now for essays

# get the POS patterns 
joined_pos, tuples = get_pos(essay_bodies)
ngrams_pos = token_patterns(joined_pos, 5, 10)

In [42]:
cr = compression_ratio(joined_pos, 'gzip')
# hs = homogenization_score(data_example, 'rougel')
# hs = homogenization_score(data_example, 'bertscore') 
nds = ngram_diversity_score(joined_pos, 4)

print(cr, nds)

5.072 0.518


| Source | NDS (Lexical) | CR-Lexical | CR-POS |
| --- | --- | --- | --- |
| Human solo essays | 2.74 | 3.06 | 5.07 | 
| Personahub-essays | 1.72 | 2.90 | 6.09 | 